# Import Dependencies.

In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd

In [3]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [4]:
# Import Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
engine = create_engine("sqlite:///hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# We can view all of the classes that automap found
# Note: outcome keys of this code is (measurement, station)
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

## Retrieve Precipitation Data

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the last data point in the database (Aug 23, 2017). 

# Note: this is used as a starting point of analyzing a 
# full year of precipitation data.
# Calculate the date one year from the last date in data set.
prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)

In [ ]:
# Retrieve the precipitation score. 

# 1. Create a variable to store the results of the query. 
results = []

# 2. To only see the most recent data, filter out the data that is older
# than a year from the last record date. 
# 2a. Add the filter() function to the existing query.
# 2b. Add .all() to the end of the existing query to extract all results
# and put them in a list. 

# Perform a query to retrieve the data and precipitation score
results = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= prev_year).all()

# 3. Check results. 
print(results)

# Exploratory Climate Analysis

In [ ]:
# Save the query results as a Pandas DataFrame. and set the index to the date column.
df = pd.DataFrame(results, columns=['date','precipitation'])

# Check df without date set as index.
#print(df)

# Set the index to the date column.
df.set_index(df['date'], inplace=True)

# Check df with date set as index.
#print(df)

# Using data as index shows duplicate date columns; 
# set df to print without index. 
print(df.to_string(index=False))

In [ ]:
# Sort the dataframe by date.
# Note: The goal is to create a plot of precipitation scores in chronological order,
# to track if the weather trend will be feasible for the investment.
df = df.sort_index()

# Check list to ensure dates are in chronological order; print without index.
print(df.to_string(index=False))

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data.
# Note: this graph shows the total precipitation per day.
df.plot()

In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data
# 1. Calculate the mean, minimum, maximum, stddev and percentiles.
# Note: the "count" is the number of times precipitation was observed. 
df.describe()

## Station Analysis

In [ ]:
# How many stations are available in this dataset?
# Note: write a query to get the number of stations 
# that are collecting precipitation data (returns 9)
session.query(func.count(Station.station)).all()


In [ ]:
# What are the most active stations?
# Note: it's important to know which stations have the most
# precipitation recordings.

# 1. Write beginnining query.
# session.query(Measurement.station, func.count(Measurement.station))


# 2. To narrow down the data by station name, add group_by()
# to the query code. 

# session.query(Measurement.station, func.count(Measurement.station)).\group_by(Measurement.station)

# 3. List the stations and the counts in descending order.
# Note: Update the code with order_by(func.count(Measurement.station).desc()).

# session.query(Measurement.station, func.count(Measurement.station)).\
# group_by(Measurement.station).order_by(func.count(Measurement.station).desc())


# 4. Add .all() function to return all the results of the query.
session.query(Measurement.station, func.count(Measurement.station)).\
group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()

# Note: the left column is the station ID, right column is for precipitation counts 
# for each station. According to the data, the most active station
# is USC00519281 (2772 counts).


## Temperature Analysis

In [ ]:
# Using the station ID from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature most active station?

# 1. Minimum, maximum, average temperatures coded using the following
# functions: func.min, func.max, func.avg.

# session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs))

# 2. Filter using the most active stations (USC00519281)

# session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)).\
# filter(Measurement.station == 'USC00519281')

# 3. Return final query results using .all() function.
session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)).\
filter(Measurement.station == 'USC00519281').all()

# Note: Minimum temp (54.0), Maximum temp (85.0), Avg temp (71.7)

In [ ]:
# Choose the station with the highest number of temperature observations.
# Note: station is same as the one for precipitation recordings
# station ID: USC00519281


# 1. Query the last 12 months of temperature observation data for this station.
# session.query(Measurement.tobs)

# 1a. Filter out all the stations except USC00519281.
# results = session.query(Measurement.tobs).\
# filter(Measurement.station == 'USC00519281')

# 1b. Filter data to consider only the most recent year, then add
# .all() function to save our results as a list. 
# Print results to check data. 

results = session.query(Measurement.tobs).\
filter(Measurement.station == 'USC00519281').\
filter(Measurement.date >= prev_year).all()
print(results)

### Convert the Temperature Observation Results to a DataFrame.

In [ ]:
# 1. Put data into a DataFrame for easier reading.
df = pd.DataFrame(results, columns=['tobs'])
#print(df)

# 2. Print with removed index column.
print(df.to_string(index=False))

### Plot the Temperature Observations

In [ ]:
# 2. Plot the results as a histogram.
# Note: a histogram is a graph made up of a range of data that
# is separated into different bins. When creating a histogram
# you need to determine how many bins are needed (ideal is 5-20 bins).
# A "good fit" of the histogram represents the data well and highlights
# areas where there is a lot of dat and areas where there isn't a 
# lot of data.

# 2a. Divide temperature observations into 12 different bins.
df.plot.hist(bins=12)
plt.tight_layout()

# Note: from the plot, we notice that the majority of observed
# temperatures are over 67 degrees. To the right. counting the 
# bins, there is about 325 days where the temp was over 67 degrees. 

In [ ]:
# Skill Drill: Adjust the number of bins. 

# Notes on number of bins as 5:

# Notes on number of bins as 20:


## Set Up Flask and Create a Route

In [ ]:
# Note: we are going to create five routes for our investors to
# clearly navigate to the analysis they want to see:
# 1. Welcome
# 2. Precipitation
# 3. Stations
# 4. Monthly Temperature
# 5. Statistics

##############################
# Steps to Take:
##############################

# 1. Install Flask.
# 2. Create a new Python file.
# 3. Import the Flask dependency.
# 4. Create a new Flask app instance.
# 5. Create Flask routes.
# 6. Run a Flask app. 

In [ ]:
# Skill Drill:
#Write a function called `calc_temps` that will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temp(start_date, end_date):
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()
print(calc_temp('2012-02-28', '2012-03-05'))

# Challenge